In [1]:
from __future__ import print_function
import torch

In [2]:
x = torch.Tensor(5, 3)
print(x)

tensor([[9.3674e-39, 1.0929e-38, 1.0469e-38],
        [8.4490e-39, 1.0102e-38, 9.0919e-39],
        [1.0102e-38, 8.9082e-39, 8.4489e-39],
        [1.0102e-38, 1.0561e-38, 1.0653e-38],
        [1.0469e-38, 9.5510e-39, 9.9184e-39]])


In [3]:
x = torch.rand(5, 3)
print(x)

tensor([[0.2348, 0.9381, 0.9667],
        [0.9308, 0.7166, 0.0126],
        [0.1401, 0.5488, 0.1028],
        [0.4698, 0.0174, 0.5735],
        [0.1757, 0.9967, 0.5858]])


In [4]:
y = torch.rand(5, 3)
print(x + y)

tensor([[1.2246, 1.7659, 1.7729],
        [1.6153, 1.6693, 0.7597],
        [0.9025, 0.5936, 0.2936],
        [1.0456, 0.6590, 1.2916],
        [0.6516, 1.9196, 0.9450]])


In [5]:
print(torch.add(x, y))

tensor([[1.2246, 1.7659, 1.7729],
        [1.6153, 1.6693, 0.7597],
        [0.9025, 0.5936, 0.2936],
        [1.0456, 0.6590, 1.2916],
        [0.6516, 1.9196, 0.9450]])


In [6]:
result = torch.Tensor(5, 3)
torch.add(x, y, out=result)
print(result)

tensor([[1.2246, 1.7659, 1.7729],
        [1.6153, 1.6693, 0.7597],
        [0.9025, 0.5936, 0.2936],
        [1.0456, 0.6590, 1.2916],
        [0.6516, 1.9196, 0.9450]])


In [7]:
# 把x加到y上
y.add_(x)
print(y)

tensor([[1.2246, 1.7659, 1.7729],
        [1.6153, 1.6693, 0.7597],
        [0.9025, 0.5936, 0.2936],
        [1.0456, 0.6590, 1.2916],
        [0.6516, 1.9196, 0.9450]])


In [8]:
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y

In [9]:
from torch.autograd import Variable

In [10]:
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [11]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [12]:
print(y.grad_fn)
print(x.grad_fn)

None


In [13]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [14]:
out.backward()

In [15]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [16]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-392.0968, -449.7450, 1590.1205], grad_fn=<MulBackward0>)


In [17]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)
print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


### 对backward()函数做验证

In [36]:
x=Variable(torch.Tensor([2,3,4,5]),requires_grad=True)
y=Variable(torch.zeros(3))

In [37]:
y[0]=x[0]*x[1]
y[1]=x[1]*x[2]
y[2]=x[2]*x[3]

In [39]:
out = y.mean()
out.backward()

In [22]:
y.backward(torch.ones(y.shape))

In [40]:
x.grad

tensor([1.0000, 2.0000, 2.6667, 1.3333])

In [29]:
y.backward(torch.Tensor([2,3,4]))
x.grad

tensor([ 6., 16., 29., 16.])

## 如果最后的out不是一个标量，是需要在backward()中放入一个和out通维度的权重向量
$$grad\_x_0=w_0*\frac{\partial y_0}{\partial x_0}+w_1*\frac{\partial y_1}{\partial x_0}+w_2*\frac{\partial y_2}{\partial x_0}=w_0*x_1$$
**同理:** 

$grad\_x_1=w_0*x_0+w_1*x_2$, 

$grad\_x_2=w_1*x_1+w_2*x_3$, 

$grad\_x_3=w_2*x_2$
## 如果最后的out是一个标量则不用

## 定义网络

In [42]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5*5 square convolution
        # kernel

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [43]:
params = list(net.parameters())
print(len(params))
for param in params:
    print(param.size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [44]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

tensor([[ 0.0184,  0.0262, -0.0530,  0.0453, -0.0703,  0.0009,  0.0197,  0.0287,
         -0.0472, -0.1309]], grad_fn=<AddmmBackward>)


In [45]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [47]:
out = net(input)
target = Variable(torch.arange(1, 11)).float()  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(out, target)
print(loss)

tensor(38.8269, grad_fn=<MseLossBackward>)


In [50]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

#in your trainning loop:
optimizer.zero_grad()  # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # does the update

In [51]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


0it [00:00, ?it/s]


  1%|▍                                                                | 1253376/170498071 [00:49<4:11:28, 11216.65it/s]


  1%|▉                                                                | 2514944/170498071 [01:15<1:05:45, 42578.43it/s]


  3%|█▊                                                                 | 4546560/170498071 [01:37<47:44, 57924.57it/s]


  4%|██▎                                                              | 6119424/170498071 [02:05<1:24:33, 32401.80it/s]


...

 64%|█████████████████████████████████████████▍                       | 108748800/170498071 [37:43<28:24, 36229.02it/s]


 64%|█████████████████████████████████████████▉                       | 109879296/170498071 [38:17<22:27, 44977.58it/s]


 65%|█████████████████████████████████████████▋                      | 111206400/170498071 [38:52<09:29, 104049.00it/s]


 68%|███████████████████████████████████████████▉                     | 115138560/170498071 [39:15<22:55, 40236.56it/s]


 68%|████████████████████████████████████████████▏                    | 115892224/170498071 [39:28<15:00, 60647.82it/s]

In [ ]:
trainset